Guided Practice Project

In [ ]:
#imports
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict



In [ ]:
class Sentiment:
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'
    POSITIVE = 'POSITIVE'

# Reviews Class
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    def distribute_evenly(self):
      negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
      positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
      neutral = list(filter(lambda x: x.sentiment == Sentiment.NEUTRAL, self.reviews))
      positive_shrunk = positive[:len(negative)]
      self.reviews = negative + positive_shrunk
      random.shuffle(self.reviews)



In [ ]:
#load in data
import json
filename = 'Books_small_10000.json'
reviews = []

with open(filename) as f:
    for line in f:
       review = json.loads(line)
       reviews.append(Review(review['reviewText'], review['overall']))







In [ ]:
# data prep
train, test = train_test_split(reviews, test_size=0.33, random_state=42)


train_container = ReviewContainer(train)
test_container = ReviewContainer(test)

train_container.distribute_evenly()
test_container.distribute_evenly()

x_train = train_container.get_text()
y_train = train_container.get_sentiment()

x_test = test_container.get_text()
y_test = test_container.get_sentiment()





In [ ]:
vectorizer = TfidfVectorizer()

train_x_vectors = vectorizer.fit_transform(x_train)
test_x_vectors = vectorizer.transform(x_test)



In [ ]:
# Linear svm

clf_svc = LinearSVC()
clf_svc.fit(train_x_vectors, y_train)
print(clf_svc.predict(test_x_vectors[0]))
print(clf_svc.score(test_x_vectors, y_test))


['POSITIVE']
0.8221153846153846


In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

clf_dt = DecisionTreeClassifier()
clf_dt.fit(train_x_vectors, y_train)
print(clf_dt.predict(test_x_vectors[0]))
print(clf_dt.score(test_x_vectors, y_test))

['NEGATIVE']
0.6682692307692307


In [ ]:
#naive bayes

clf_nb = MultinomialNB()
clf_nb.fit(train_x_vectors, y_train)
print(clf_nb.predict(test_x_vectors[0]))
print(clf_nb.score(test_x_vectors, y_test))

['POSITIVE']
0.8125


In [ ]:
#f1 score
from sklearn.metrics import f1_score
print(f1_score(y_test, clf_svc.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(y_test, clf_dt.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(y_test, clf_nb.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))


[0.82629108 0.81773399]
[0.66009852 0.67605634]
[0.79144385 0.82969432]


In [ ]:
#test with chosen model linear svc
test_set = ['very fun', 'This Sucks', 'horrible waste of time']
new_test = vectorizer.transform(test_set)
print(clf_svc.predict(new_test))


['POSITIVE' 'POSITIVE' 'NEGATIVE']


In [ ]:
#grid search improvement
from sklearn import svm

parameters = {'C':[1, 10]}
svc = LinearSVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, y_train)
print(clf.score(test_x_vectors, y_test))

0.8221153846153846


In [ ]:
#save model
import pickle

with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [ ]:
#load model
with open('sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

print(x_test[6])
loaded_clf.predict(test_x_vectors[6])

I started this because i was searching for something personal, something that would touch me and what I was experiencing at the time.  Unfortunately, this book did not touch me nor changed my life forever.  To be fair, however, I may give it another chance.


array(['NEGATIVE'], dtype='<U8')